In [ ]:
import triton
import SupertracePybind as Supertrace
from supertrace_util import compatibleProcessing, initTritonCtxEnv, mergeRepeatIns, checkIndirectIns, getMemoryMapByAddr

In [ ]:
tracepath = "11111111.trace64"

trace = Supertrace.parse_x64dbg_trace(tracepath)
record = trace.getRecord()
print(f"trace instruction num: {len(record)}")

In [ ]:
ctx = triton.TritonContext()
ctx.setArchitecture(triton.ARCH.X86_64)
ctx.setMode(triton.MODE.ALIGNED_MEMORY, True)
ctx.setMode(triton.MODE.AST_OPTIMIZATIONS, True)
ctx.setMode(triton.MODE.CONSTANT_FOLDING, True)
ctx.setMode(triton.MODE.ONLY_ON_SYMBOLIZED, True)
# ctx.setMode(triton.MODE.MEMORY_ARRAY, True)
# ctx.setMode(triton.MODE.SYMBOLIZE_LOAD, True)
# ctx.setMode(triton.MODE.SYMBOLIZE_STORE, True)
ctx.setMode(triton.MODE.SYMBOLIZE_INDEX_ROTATION, True)
ctx.setAstRepresentationMode(triton.AST_REPRESENTATION.PYTHON)
astctx = ctx.getAstContext()

In [ ]:
record = mergeRepeatIns(ctx, record, True)
print(f"after mergeing 'rep' instructions, trace instruction num: {len(record)}")

In [ ]:
threads = trace.user.meta.getThreads()
for th in threads:
    if th.id == record[0].thread_id:
        main_thread = th
        break
print(f"main thread id: {main_thread.id} ({hex(main_thread.id)})")
print(f"teb: {hex(main_thread.teb)}")
initTritonCtxEnv(ctx, record[0], main_thread.teb)

In [ ]:
for i, ins in enumerate(record):
    ttins = triton.Instruction()
    ttins.setAddress(ins.ins_address)
    ttins.setOpcode(ins.bytes)
    ctx.disassembly(ttins)
    if (i + 1 >= len(record)): nextIns = None
    else: nextIns = record[i + 1]

    if (ins.dbg_id == 0):
        symvar = ctx.symbolizeRegister(ctx.registers.ecx, "user")

    compatibleProcessing(ctx, ttins, ins, nextIns, True, False)

    if (checkIndirectIns(ttins)):
        pcExpr = ctx.getSymbolicRegister(ctx.registers.rip)
        if ((pcExpr is None) or (not pcExpr.isSymbolized())):
            continue
        pcAst = pcExpr.getAst()
        unrollPcAst = astctx.unroll(pcAst)
        final = unrollPcAst

        mainSat = (final != astctx.bv(final.evaluate(), final.getBitvectorSize()))

        satProve = mainSat

        model = ctx.getModel(satProve, True)
        status = model[1]
        if (status == triton.SOLVER_STATE.SAT):
            print(f"[{hex(ins.dbg_id)}] 已证明成功! 求解时间: {model[2]} 路径谓词: {model[0]}")
        elif (status == triton.SOLVER_STATE.UNSAT):
            print(f"[{hex(ins.dbg_id)}] 证明失败 求解时间: {model[2]}")
        elif (status == triton.SOLVER_STATE.TIMEOUT):
            print(f"[{hex(ins.dbg_id)}] 证明过程已超时")
        elif (status == triton.SOLVER_STATE.OUTOFMEM):
            print(f"[{hex(ins.dbg_id)}] 证明过程内存消耗殆尽")
        elif (status == triton.SOLVER_STATE.UNKNOWN):
            print(f"[{hex(ins.dbg_id)}] 证明过程发生未知错误: {status}")